In [1]:
import os
import numpy as np
from PIL import Image, ImageDraw
from pickle import load
import NNfunctions as NN
from keras.models import load_model

root_dir = 'C:/Users/Varun/Desktop/VOCdevkit/VOC2010'
img_dir = os.path.join(root_dir, 'JPEGImages')

Using TensorFlow backend.


In [2]:
x_master, y_master, z_master = NN.get_dataset('C:/Users/Varun/Desktop/AI-ESA/preprocessing')

[4744, 4744, 4744]


In [3]:
indices = np.arange(x_master.shape[0])
x_master = x_master[indices]
y_master = y_master[indices]
z_master = z_master[indices]

NB_TRAIN = 3500

x_train = np.array(x_master[:NB_TRAIN])
y_train = np.array(y_master[:NB_TRAIN])
z_train = np.array(z_master[:NB_TRAIN])

print(x_train.shape, z_train.shape)

(3500, 128, 128, 3) (3500, 15)


In [12]:
from keras import applications
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D

base_model = applications.VGG16(weights = 'imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(15, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

model.summary()

model.fit(x_train, z_train,
          batch_size = 50,
          epochs = 8,
          validation_split = 0.2)
model.save('transfer_learning_model.h5')
del model

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [13]:
model = load_model('transfer_learning_model.h5')
print(NN.predict_class(model, os.path.join(img_dir, '2007_000032.jpg')))
del model

[ 0.08621018  0.00348278  0.04653129  0.07229409  0.47128791  0.04046512
  0.02108211  0.00775007  0.02766068  0.13710414  0.03745021  0.02411136
  0.00999911  0.01100108  0.00356989]
aeroplane


In [16]:
model = load_model('transfer_learning_model.h5')
x_test, z_test = x_master[-1000:], z_master[-1000:]
print(model.evaluate(x_test, z_test, batch_size = 10))
del model

1000/1000 [==============================] - 29s 29ms/step
[1.2456729918718339, 0.58500000596046453]


In [15]:
model = load_model('transfer_learning_model.h5')
NN.make_confusion_matrix(model, x_test, z_test)
del model

CONFUSION MATRIX
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
[[140   2   8   9   4   0   0   6  15   0   0   1   2   0   2]
 [  2  19   0   2   0   0   0   0   0   0   0   0   0   0   1]
 [ 14   0  47   1   8   0   0  21  10   0   0   0   0   0   0]
 [  1   0   2  41   2   0   0   3   0   0   2   0   0   0   0]
 [  0   0   4  12  74   0   0   0   0   0   0   1   0   0   0]
 [  9   0   3   3   0   9   0   7   5   0   0   1   0   0   0]
 [  3   0   3   2   1   4   0   1  13   0   0   0   0   0   0]
 [ 18   0   9   4   0   0   0  93  35   0   0   0   1   0   1]
 [ 15   0  10   1   4   1   0  19  88   1   0   0   0   0   0]
 [  2   0   4  15   8   0   0   0   1   2   0   0   0   0   0]
 [  5   0   1  11   6   0   0   0   1   1  30   0   1   0   0]
 [  7   0   1   0   2   0   0   0   1   0   0  20   0   0   0]
 [  8   0   0   1   1   0   0   0   6   0   0   0   8   1   1]
 [  4   2   3   1   1   0   0   1   0   0   0   0   0  10   0]
 [  9   0   0   0   0   0   0   0   0   0   0   0   0 